# Investment Assignment

# Checkpoint 1: Data Cleaning


In [ ]:
#Loading the companies and rounds data

import numpy as np
import pandas as pd

companies = pd.read_csv('companies.csv')
rounds2 = pd.read_csv('rounds2.csv')

In [ ]:
companies.head()

In [ ]:
rounds2.head()

In [ ]:
#Observation : duplicates of company names found with change in the letter case 

In [ ]:
companies.info()

In [ ]:
rounds2.info()

In [ ]:
#checking for number of unique company names in rounds2
rounds2.company_permalink.value_counts()

In [ ]:
#checking for number of unique company names in companies
#assumption: all company names listed are unique in companies data file, no duplicates
companies.permalink.value_counts()

In [ ]:
#Observation : The encoding is not correct
# Checking for correct encoding using chardet library

import chardet

with open("rounds2.csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))

# check what the character encoding might be
print(result)

In [ ]:
#importing data with correct encoding 'ISO-8859-1'
rounds2 = pd.read_csv('rounds2.csv', encoding = 'ISO-8859-1')
companies = pd.read_csv('companies.csv', encoding = 'ISO-8859-1')

In [ ]:
#changing the letter case to lower to eliminate duplicate values by using lower() method to eliminte duplicates
rounds2.company_permalink = rounds2.company_permalink.str.lower()
rounds2.head()

In [ ]:
#checking for number of unique company names in rounds2 dataframe by using unique() method
len(rounds2.company_permalink.unique())

In [ ]:
#changing the letter case to lower for correct merging of databy using lower() method to eliminte duplicates
companies.permalink = companies.permalink.str.lower()
companies.head()

In [ ]:
#checking for number of unique company names in companies dataframe by using unique() method
len(companies.permalink.unique())

In [ ]:
#Merge the two data frames rounds2 and companies into master_frame dataframe
master_frame = rounds2.merge(companies, how = 'left', left_on = 'company_permalink', right_on = 'permalink')
master_frame.head()


In [ ]:
#Number of observations in master_frame dataframe
master_frame.shape

In [ ]:
#Checking for unmatched rows after merging
master_frame[master_frame.permalink.isna()]

In [ ]:
# Observation : all the above companies are available in companies data but with a slight change in a single character in the name. 

#The below 2 companies are the reason for more company names in the round2 data. 
#These companies have appeared with 2 variants of names in the rounds2 data.

# 58473 /organization/magnet-tech-ç£ç³ç§æ
# 109969 /organization/weiche-tech-åè½¦ç§æ


# Checkpoint 2: Funding Type Analysis

In [ ]:
#Checking for different funding round types using value_counts() method
master_frame.funding_round_type.value_counts()

In [ ]:
#Filtering the data with 4 funding round types 'venture', 'angel', 'seed', 'private_equity'
master_frame = master_frame[master_frame.funding_round_type.isin(['venture','angel','seed','private_equity'])]
master_frame.funding_round_type.value_counts()

In [ ]:
master_frame.shape

In [ ]:
#Removing rows with no investment amount values in the dataframe
master_frame = master_frame[~master_frame.raised_amount_usd.isna()]
master_frame.shape

In [ ]:
#checking the number of investments funding type wise after removal of nulls
master_frame.funding_round_type.value_counts()

In [ ]:
#Removing redundant rows from the dataframe
master_frame.drop(['funding_round_permalink', 'funding_round_code', 'funded_at', 'permalink', 'homepage_url',
                  'state_code', 'region', 'city', 'founded_at'], axis = 1, inplace = True)
master_frame.shape

In [ ]:
#importing libraries for plotting and ignoring the warnings
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Analysing the spread of invested amount by grouping the data based on funding type
master_frame.groupby("funding_round_type")['raised_amount_usd'].describe()

In [ ]:
# Boxplot of invested amount grouped by funding type
plt.figure(figsize= (10,10))
ax = sns.boxplot(data = master_frame, x = 'funding_round_type', y = 'raised_amount_usd')
plt.ylim(0, 200000000)
y_ticks = np.linspace(0, 200000000, 41)
plt.yticks(y_ticks)
ax.set(xlabel='Funding round type', ylabel='Investment amount', title = 'Funding round type vs Investment amount')
plt.show()

In [ ]:
#plot showing the representative amount of investment in each funding type
plt.figure(figsize= (10,7))
ax = sns.barplot(data = master_frame, x = 'funding_round_type', y = 'raised_amount_usd', estimator = np.median)
ax.set(xlabel='Funding round type', ylabel='Representative amount of investment', title = 'Funding round type vs Representative amount of investment')
plt.show()

In [ ]:
#Function to remove outliers from the data based on IQR formula
def remove_outliers(df):
    Q1=df.quantile(0.25)
    Q3=df.quantile(0.75)
    IQR=Q3-Q1
    df=df[~((df<(Q1-1.5*IQR)) | (df>(Q3+1.5*IQR)))]
    return df

In [ ]:
#Removing the outliers from the data based on funding type and analysing the spread of data 
for i in ['angel','private_equity','seed','venture']:
    df = master_frame[master_frame.funding_round_type == i]
    df = remove_outliers(df)
    print(f'{i} : {df.raised_amount_usd.describe()}\n')
    

In [ ]:
#Boxplot of funding type - venture after removal of outliers
sns.boxplot(data = df, x = 'funding_round_type', y = 'raised_amount_usd')
plt.show()

In [ ]:
#Observation : Funding type Venture is the suitable one for spark funds to invest before and after the removal of outliers. 
#Not removing the outliers as we don't know the exact reason for the outliers
#Median value of the data without removing outliers : Representative Values of Investments for Each of these Funding Types
master_frame.groupby("funding_round_type")['raised_amount_usd'].median()

# Checkpoint 3: Country Analysis

In [ ]:
#Creating new data frame with only venture as funding type
df_venture = master_frame[master_frame.funding_round_type == 'venture']
df_venture.shape

In [ ]:
df_venture.head()

In [ ]:
#top nine countries which have received the highest total funding on the funding type : venture
df_countries = df_venture.groupby('country_code')['raised_amount_usd'].sum().sort_values(ascending = False)
df_countries[0:9]

In [ ]:
#Creating list of top 9 counties names
countries_list = df_countries[0:9].index.to_list()
countries_list

In [ ]:
#Creating 'top9' dataframe with the top nine countries which have received the highest total funding  
top9 = df_venture[df_venture.country_code.isin(countries_list)]
top9.shape

In [ ]:
#top 9 countries against the total amount of investments of funding type venture in descending order
pivot = top9.pivot_table( index = 'country_code', values = 'raised_amount_usd', aggfunc = np.sum).sort_values('raised_amount_usd', ascending = False)
pivot.plot(kind = 'bar', figsize = (10,7), xlabel = 'Country', ylabel = 'Total amount of investments', title = "Country vs Total amount of investments")
plt.show()

In [ ]:
#Observation : top three English-speaking countries in the data frame top9
#USA
#GBR
#IND

# Checkpoint 4: Sector Analysis 1

In [ ]:
#Importing mapping.csv file into new dataframe
mapping = pd.read_csv('mapping.csv')
mapping.head()

In [ ]:
#Dropping the redundant column 'Blanks'
mapping.drop('Blanks', axis = 1, inplace = True)
mapping.head()

In [ ]:
#Separating the value variables from the mappings dataframe and forming a lsit
value_vars = list(mapping.columns[1:])
value_vars

In [ ]:
# Transforming the wide dataframe into long dataframe by using melt method
long_mapping = pd.melt(mapping, id_vars = ['category_list'], value_vars = value_vars)
long_mapping

In [ ]:
#Filtering the data for correct mappings by removing zero values in the data
long_mapping = long_mapping[~(long_mapping.value == 0)]
long_mapping

In [ ]:
#Dropping the value column and renaming the variable column to main_sector
long_mapping = long_mapping.drop('value', axis = 1)
long_mapping = long_mapping.rename(columns = {'variable' : 'main_sector'})
long_mapping

In [ ]:
#Removing the rows with null values in category_list column in top9 dataframe
top9 = top9[~(top9.category_list.isna())]
top9.shape

In [ ]:
#Extract the primary sector of each category list from the category_list column
top9.category_list = top9.category_list.apply(lambda x : x.split("|",1)[0])
top9.sample(10)

In [ ]:
#merged data frame with each primary sector mapped to its main sector
df_final_mapping = pd.merge(top9, long_mapping, how = 'left', on = 'category_list')
df_final_mapping.head()

# Checkpoint 5: Sector Analysis 2

In [ ]:
#Taking rows whose investment amount is falling within the 5-15 million USD range
df_final_mapping = df_final_mapping[df_final_mapping.raised_amount_usd.between(5000000, 15000000, inclusive=True)]

#Removing rows with null values in main_sector column in the merged dataframe
df_final_mapping = df_final_mapping[~(df_final_mapping.main_sector.isna())]

#Filtering the data using top 3 English speaking countries with highest investment amount
df_final_mapping = df_final_mapping[df_final_mapping.country_code.isin(['USA', 'GBR', 'IND'])]

df_final_mapping

In [ ]:
#Bar plot showing the number of investments in the all 8 sectors of the top 3 countries : Normalised data
final_plot = df_final_mapping.groupby(['country_code'])['main_sector'].value_counts(normalize=True)*100
final_plot.plot(kind = 'bar', figsize = (10,7), xlabel = 'Country, Main sector', ylabel = 'Percentage of investments of each sector country-wise', 
               title = 'Country, Main sector vs Percentage of investments of each sector country-wise')
plt.show()

In [ ]:
#Creating individual data frames D1, D2 and D3 for top 3 English speaking countries with highest investment amount
D1 = df_final_mapping[df_final_mapping.country_code == 'USA']
D2 = df_final_mapping[df_final_mapping.country_code == 'GBR']
D3 = df_final_mapping[df_final_mapping.country_code == 'IND']

In [ ]:
#Total number of investments in each data frame country wise
print(D1.shape, D2.shape, D3.shape)

In [ ]:
#Total amount of investment (USD) country wise
print(D1.raised_amount_usd.sum(), D2.raised_amount_usd.sum(), D3.raised_amount_usd.sum())

In [ ]:
#Number of investments sector wise in USA in descending order
D1.main_sector.value_counts()

In [ ]:
#Number of investments sector wise in GBR in descending order
D2.main_sector.value_counts()

In [ ]:
#Number of investments sector wise in IND in descending order
D3.main_sector.value_counts()

In [ ]:
#Total amount of investment sector wise in USA in ascending order
D1.groupby("main_sector")['raised_amount_usd'].sum().sort_values()

In [ ]:
#Total amount of investment sector wise in GBR in ascending order
D2.groupby("main_sector")['raised_amount_usd'].sum().sort_values()

In [ ]:
#Total amount of investment sector wise in IND in ascending order
D3.groupby("main_sector")['raised_amount_usd'].sum().sort_values()

In [ ]:
#Total investment in each company in Others sector in USA in ascending order
D1[D1.main_sector == 'Others'].groupby('company_permalink')['raised_amount_usd'].sum().sort_values()

In [ ]:
#Total investment in each company in Others sector in GBR in ascending order
D2[D2.main_sector == 'Others'].groupby('company_permalink')['raised_amount_usd'].sum().sort_values()

In [ ]:
#Total investment in each company in Others sector in IND in ascending order
D3[D3.main_sector == 'Others'].groupby('company_permalink')['raised_amount_usd'].sum().sort_values()

In [ ]:
#Total investment in each company in Cleantech / Semiconductors sector in USA in ascending order
D1[D1.main_sector == 'Cleantech / Semiconductors'].groupby('company_permalink')['raised_amount_usd'].sum().sort_values()

In [ ]:
#Total investment in each company in Cleantech / Semiconductors sector in GBR in ascending order
D2[D2.main_sector == 'Cleantech / Semiconductors'].groupby('company_permalink')['raised_amount_usd'].sum().sort_values()

In [ ]:
#Total investment in each company in News, Search and Messaging sector in IND in ascending order
D3[D3.main_sector == 'News, Search and Messaging'].groupby('company_permalink')['raised_amount_usd'].sum().sort_values()

# Checkpoint 6: Plots

In [ ]:
#1)plot showing the representative amount of investment in each funding type
plt.figure(figsize= (10,7))
ax = sns.barplot(data = master_frame, x = 'funding_round_type', y = 'raised_amount_usd', estimator = np.median)
ax.set(xlabel='Funding round type', ylabel='Representative amount of investment', title = 'Funding round type vs Representative amount of investment')
plt.show()

In [ ]:
#2)top 9 countries against the total amount of investments of funding type venture in descending order
pivot = top9.pivot_table( index = 'country_code', values = 'raised_amount_usd', aggfunc = np.sum).sort_values('raised_amount_usd', ascending = False)
pivot.plot(kind = 'bar', figsize = (10,7), xlabel = 'Country', ylabel = 'Total amount of investments', title = "Country vs Total amount of investments")
plt.show()

In [ ]:
#3)Bar plot showing the number of investments in the all 8 sectors of the top 3 countries : Normalised data
final_plot = df_final_mapping.groupby(['country_code'])['main_sector'].value_counts(normalize=True)*100
final_plot.plot(kind = 'bar', figsize = (10,7), xlabel = 'Country, Main sector', ylabel = 'Percentage of investments of each sector country-wise', 
               title = 'Country, Main sector vs Percentage of investments of each sector country-wise')
plt.show()